In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/semeval-18/SemEval2018-T3_input_test_taskA.txt
/kaggle/input/semeval-18/SemEval2018-T3_gold_test_taskA_emoji.txt
/kaggle/input/semeval-18/SemEval2018-T3-train-taskA.txt


In [2]:
pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 11.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=9304be43bb8c8e3befd9003ebff96dca3f822846a333e036f7a2eb82bbfc7539
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Create an empty list to store the data
data_list = []

# Open the .txt file for reading
with open('/kaggle/input/semeval-18/SemEval2018-T3-train-taskA.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an index
index = 0

for index, line in enumerate(lines):
    if index == 0:  # Skip the first line (index 0)
        continue
    
    tweet_index, label, tweet_text = line.strip().split('\t')
    data_dict = {
        'idx': tweet_index,
        'sentence': tweet_text,
        'label': int(label)  # Convert label to an integer if it's not already
    }
    data_list.append(data_dict)



In [4]:
# Create an empty list to store the data
data_list_test = []

# Open the .txt file for reading
with open('/kaggle/input/semeval-18/SemEval2018-T3_gold_test_taskA_emoji.txt', 'r') as file:
    # Read the lines from the file
    lines = file.readlines()

# Initialize an index
index = 0

for index, line in enumerate(lines):
    if index == 0:  # Skip the first line (index 0)
        continue
    
    tweet_index, label, tweet_text = line.strip().split('\t')
    data_dict_test = {
        'idx': tweet_index,
        'sentence': tweet_text,
        'label': int(label)  # Convert label to an integer if it's not already
    }
    data_list_test.append(data_dict_test)


REMOVING HYPERLINKS

In [5]:
import re

# Function to remove hyperlinks from a text
def remove_hyperlinks(text):
    return re.sub(r'http\S+', '', text)

# Iterate through the list of dictionaries and clean the 'sentence' field
for data in data_list:
    data['sentence'] = remove_hyperlinks(data['sentence'])
    
for data in data_list_test:
    data['sentence'] = remove_hyperlinks(data['sentence'])


Remove hashtasg and keep the word

In [6]:
# Function to remove '#' from hashtags in a text
def remove_hashtag_symbol(text):
    return re.sub(r'#', '', text)

# Iterate through the list of dictionaries and clean the 'sentence' field
for data in data_list:
    data['sentence'] = remove_hashtag_symbol(data['sentence'])
    
for data in data_list_test:
    data['sentence'] = remove_hashtag_symbol(data['sentence'])


remove mentions

In [7]:
# Function to remove words starting with '@' in a text
def remove_mentions(text):
    return ' '.join(word for word in text.split() if not word.startswith('@'))

# Iterate through the list of dictionaries and clean the 'sentence' field
for data in data_list:
    data['sentence'] = remove_mentions(data['sentence'])
    
for data in data_list_test:
    data['sentence'] = remove_mentions(data['sentence'])


In [8]:
pip install demoji


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import demoji

# Download the demoji library's emoji data
demoji.download_codes()

# Function to remove emojis from a text
def remove_emojis(text):
    return demoji.replace(text, '')

# Iterate through the list of dictionaries and clean the 'sentence' field
for data in data_list_test:
    data['sentence'] = remove_emojis(data['sentence'])


/tmp/ipykernel_32/3179315310.py:4: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [10]:
data_list_test[:10]

[{'idx': '1',
  'sentence': 'Can U Help?||More conservatives needed on TSU + get paid 4 posting stuff like this!||YOU $ can go to',
  'label': 0},
 {'idx': '2',
  'sentence': 'Just walked in to Starbucks and asked for a "tall blonde" Hahahaha irony',
  'label': 1},
 {'idx': '3', 'sentence': 'NOT GONNA WIN', 'label': 0},
 {'idx': '4',
  'sentence': 'He is exactly that sort of person. Weirdo!',
  'label': 0},
 {'idx': '5',
  'sentence': "So much sarcasm at work mate 10/10 boring 100% dead mate full on shit absolutely sleeping mate can't handle the sarcasm",
  'label': 1},
 {'idx': '6', 'sentence': 'Corny jokes are my absolute favorite', 'label': 0},
 {'idx': '7',
  'sentence': 'People complain about my backround pic and all I feel is like "hey don\'t blame me, Albert E might have spoken those words" sarcasm life',
  'label': 1},
 {'idx': '8', 'sentence': 'Darn, my sock joke needs fixing?', 'label': 0},
 {'idx': '9',
  'sentence': "if Christian expects Fifa to sleep in my bed with me toni

In [11]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 74 not upgraded.


In [13]:
dataset_id = "sst2"
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [14]:
from datasets import load_dataset

dataset = load_dataset(dataset_id)
dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

In [16]:
dataset["test"][0:10]

{'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'sentence': ['uneasy mishmash of styles and genres .',
  "this film 's relationship to actual tension is the same as what christmas-tree flocking in a spray can is to actual snow : a poor -- if durable -- imitation .",
  'by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .',
  'director rob marshall went out gunning to make a great one .',
  'lathan and diggs have considerable personal charm , and their screen rapport makes the old story seem new .',
  'a well-made and often lovely depiction of the mysteries of friendship .',
  "none of this violates the letter of behan 's book , but missing is its spirit , its ribald , full-throated humor .",
  "although it bangs a very cliched drum at times , this crowd-pleaser 's fresh dialogue , energetic music , and good-natured spunk are often infectious .",
  'it is not a mass-market entertainment but an uncompromising attempt by one artist to think about 

In [17]:
from datasets import Dataset, DatasetDict

In [18]:
dataset_dict = DatasetDict({
    "train": Dataset.from_dict({key: [item[key] for item in data_list] for key in data_list[0]}),
    "test" : Dataset.from_dict({key: [item[key] for item in data_list_test] for key in data_list_test[0]})
})

In [19]:
dataset_dict["train"][0:10]

{'idx': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'sentence': ['Sweet United Nations video. Just in time for Christmas. imagine NoReligion',
  "We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing ;)",
  'Hey there! Nice to see you Minnesota/ND Winter Weather',
  "3 episodes left I'm dying over here",
  '"I can\'t breathe!" was chosen as the most notable quote of the year in an annual list released by a Yale University librarian',
  "You're never too old for Footie Pajamas.",
  'Nothing makes me happier then getting on the highway and seeing break lights light up like a Christmas tree..',
  '4:30 an opening my first beer now gonna be a long night/day',
  "do you think you would support a guy who knocked out your daughter? Rice doesn't deserve support.",
  'You are not allowed to open that until Christmas day!'],
 'label': [1, 1, 1, 0, 1, 0, 1, 0, 0, 0]}

In [20]:
from setfit import sample_dataset

train_dataset = dataset_dict["train"]
train_dataset[0:10]

{'idx': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'sentence': ['Sweet United Nations video. Just in time for Christmas. imagine NoReligion',
  "We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing ;)",
  'Hey there! Nice to see you Minnesota/ND Winter Weather',
  "3 episodes left I'm dying over here",
  '"I can\'t breathe!" was chosen as the most notable quote of the year in an annual list released by a Yale University librarian',
  "You're never too old for Footie Pajamas.",
  'Nothing makes me happier then getting on the highway and seeing break lights light up like a Christmas tree..',
  '4:30 an opening my first beer now gonna be a long night/day',
  "do you think you would support a guy who knocked out your daughter? Rice doesn't deserve support.",
  'You are not allowed to open that until Christmas day!'],
 'label': [1, 1, 1, 0, 1, 0, 1, 0, 0, 0]}

In [21]:
eval_dataset = dataset_dict["test"]

In [22]:
from setfit import SetFitModel

num_classes = len(train_dataset.unique("label"))
# Assuming train_dataset is a list of dictionaries and 'label' is the key for the label
# unique_labels = set(data['label'] for data in train_dataset)

# # Calculate the number of unique classes
# num_classes = len(unique_labels)
model = SetFitModel.from_pretrained(model_id, use_differentiable_head=True, head_params={"out_features": num_classes})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [23]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=20,
    column_mapping={"sentence": "text", "label": "label"},
)

In [24]:
trainer.freeze()
trainer.train(body_learning_rate=1e-5, num_epochs=3)

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 153360
  Num epochs = 3
  Total optimization steps = 28755
  Total train batch size = 16


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9585 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9585 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Iteration:   0%|          | 0/9585 [00:00<?, ?it/s]

In [26]:
trainer.unfreeze(keep_body_frozen=True)
trainer.train(learning_rate=1e-2, num_epochs=25)

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

In [27]:
trainer.evaluate()


Applying column mapping to evaluation dataset
***** Running evaluation *****


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

{'accuracy': 0.7346938775510204}

In [37]:
model._save_pretrained('save_directory')

In [34]:
test_pred = trainer.predict(eval_dataset)

AttributeError: 'SetFitTrainer' object has no attribute 'predict'